In [313]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
import pandas as pd
import time
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func


In [314]:
executable_path = {'executable_path': GeckoDriverManager().install()}
browser = Browser('firefox', **executable_path, headless=False)



====== WebDriver manager ======
Current firefox version is 95.0.2
Get LATEST driver version for 95.0.2
Driver [C:\Users\matth\.wdm\drivers\geckodriver\win64\v0.30.0\geckodriver.exe] found in cache


In [315]:
#url = "https://www.linkedin.com/jobs/search?keywords=Data%20Analyst&location=Kansas%20City%2C%20MO&geoId=104061291&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0"

In [316]:
url = "https://www.linkedin.com/jobs/search?keywords=Data%20Analyst&position=1&pageNum=0"

In [317]:
search = ({"location_id": 1000, "city":"Kansas City, Missouri, United States"}
    , {"location_id": 1001, "city": "Austin, Texas, United States"})

In [318]:
location_df = pd.DataFrame.from_dict(search)

In [319]:
browser.visit(url)
time.sleep(5)

browser.fill("location", search[0]["city"])
time.sleep(5)
browser.find_by_xpath('/html/body/div[1]/header/nav/section/section[2]/form/button').click()





In [320]:
html = browser.html
jobs_soup = soup(html, 'html.parser')

In [321]:
no_jobs = jobs_soup.find('span', class_='results-context-header__job-count').text
no_jobs_int = int(no_jobs.replace('+', '').replace(',', ''))

In [322]:
i = 2
while i <= int(no_jobs_int/25)+1:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    i = i + 1
    try:
        browser.find_by_xpath('/html/body/div[1]/div/main/section[2]/button').click()
        time.sleep(2)
    except:
        pass
        time.sleep(2)         


In [323]:
html = browser.html
jobs_soup = soup(html, 'html.parser')

In [324]:
job_list = jobs_soup.find('ul', class_='jobs-search__results-list')


In [325]:
jobs = job_list.find_all('li')

In [326]:
job_id= []
job_title = []
company_name = []
location = []
date = []
job_link = []

for job in jobs:
    try:
        job_id0 = job.div['data-row']
        job_id.append(job_id0)
    except:
        job_id.append("no id")
    try:
        job_title0 = job.find('h3', class_='base-search-card__title').get_text()
        job_title.append(job_title0.replace('\n', '').strip())
    except:
        job_title.append("no title")
    try:
    
        company_name0 = job.find('h4', class_='base-search-card__subtitle').get_text()
        company_name.append(company_name0.replace('\n', '').strip())
    except:
        company_name.append("no name")    

    try:

        location0 = job.find('span', class_='job-search-card__location').get_text()
        location.append(location0.replace('\n', '').strip())
    except:
        location.append("no location") 

    try:
        
        date0 = job.time['datetime']
        date.append(date0.replace('\n', '').strip())
    except:
        date.append("no date") 
            
    try:
        
        job_link0 = job.find('a', class_='base-card__full-link')['href']
        job_link.append(job_link0.replace('\n', '').strip())
    except:
        job_link.append("no link") 
        

    # except:
    #     pass


In [327]:
job_df = pd.DataFrame({'CompanyName': company_name, 'JobTitle': job_title, 'Location': location, 'DateListed': date, 'JobLink': job_link, 'LocationSearch': search[0]["city"]}, index=job_id)


In [328]:
job_df

,CompanyName,JobTitle,Location,DateListed,JobLink,LocationSearch
1,AUX Partners,Data Analyst,Kansas City Metropolitan Area,2021-12-21,https://www.linkedin.com/jobs/view/data-analys...,"Kansas City, Missouri, United States"
2,SelectQuote Insurance Services,Marketing Data Analyst,Kansas City Metropolitan Area,2021-12-02,https://www.linkedin.com/jobs/view/marketing-d...,"Kansas City, Missouri, United States"
3,Cerner Corporation,Data Analyst I,"Kansas City, KS",2021-12-17,https://www.linkedin.com/jobs/view/data-analys...,"Kansas City, Missouri, United States"
4,Lockton Companies,Pharmacy Data Analyst,"Kansas City, MO",2021-12-05,https://www.linkedin.com/jobs/view/pharmacy-da...,"Kansas City, Missouri, United States"
5,Cloudflare,Data Analyst,"Kansas City, MO",2021-12-11,https://www.linkedin.com/jobs/view/data-analys...,"Kansas City, Missouri, United States"
...,...,...,...,...,...,...
996,"Vorys, Sater, Seymour and Pease LLP",Data Analyst,"Columbus, OH",2021-12-14,https://www.linkedin.com/jobs/view/data-analys...,"Kansas City, Missouri, United States"
997,Equifax,Data Quality Analyst,"St Louis, MO",2021-12-14,https://www.linkedin.com/jobs/view/data-qualit...,"Kansas City, Missouri, United States"
998,Cast & Crew,Data Analyst,"Burbank, CA",2021-12-11,https://www.linkedin.com/jobs/view/data-analys...,"Kansas City, Missouri, United States"
999,Tata Consultancy Services,Data Analyst,"San Jose, CA",2021-11-19,https://www.linkedin.com/jobs/view/data-analys...,"Kansas City, Missouri, United States"


In [329]:
jd = []
title = []
company = []
location_2 = []

for item in range(len(jobs)):
    # clicking job to view job details
    try:
        job_click_path = f'/html/body/div[1]/div/main/section[2]/ul/li[{item+1}]/div/a'
        job_click = browser.find_by_xpath(job_click_path).click()
        time.sleep(1)
    except:
        pass
    jd_path = '/html/body/div[1]/div/section/div[2]/div[1]/section[1]/div/div[2]'
    jd_path2 = '.show-more-less-html__markup'
    show_more = 'Show More'
    
    try: 

        browser.find_by_css(".show-more-less-html__button--more").click()
        time.sleep(1)
        jd0 = browser.find_by_css(jd_path2).text
        jd.append(jd0)
    except:
        jd.append("Job Description Not Availalbe")
    try: 

        job_title_path = '/html/body/div[1]/div/section/div[2]/section/div/div[1]/div/a/h2'
        title0 = browser.find_by_xpath(job_title_path).text
        title.append(title0)
    except:
        title.append("Job Title Not Availalbe")        
    try: 

        company_path = '/html/body/div[1]/div/section/div[2]/section/div/div[1]/div/h4/div[1]/span[1]/a'
        company0 = browser.find_by_xpath(company_path).text
        company.append(company0)
    except:
        company.append("Job Company Not Availalbe")
    try: 

        location_path = '/html/body/div[1]/div/section/div[2]/section/div/div[1]/div/h4/div[1]/span[2]'
        location0 = browser.find_by_xpath(location_path).text
        location_2.append(location0)
    except:
        location_2.append("Job Location Not Availalbe")    


In [330]:
job_df2 = pd.DataFrame({'Company': company, 'Location': location_2, 'JobTitle': title, "JobDesc": jd})
job_df2

,Company,Location,JobTitle,JobDesc
0,AUX Partners,Kansas City Metropolitan Area,Data Analyst,*Entry level position. Must be located or will...
1,SelectQuote Insurance Services,Kansas City Metropolitan Area,Marketing Data Analyst,"Bachelor's Degree in Finance, Marketing or Bus..."
2,Cerner Corporation,"Kansas City, KS",Data Analyst I,This role is a member of the Ascension Dedicat...
3,Lockton Companies,"Kansas City, MO",Pharmacy Data Analyst,Lockton is a global professional services firm...
4,Cloudflare,"Kansas City, MO",Data Analyst,"About Us\n\nAt Cloudflare, we have our eyes se..."
...,...,...,...,...
990,"Vorys, Sater, Seymour and Pease LLP","Columbus, OH",Data Analyst,Position Summary:\nVorys eControl provides int...
991,Equifax,"St Louis, MO",Data Quality Analyst,"Identify and research client issues, which cha..."
992,Cast & Crew,"Burbank, CA",Data Analyst,About Cast & Crew\n\nWe enable great content. ...
993,Tata Consultancy Services,"San Jose, CA",Data Analyst,"As a Data Analyst, You Will Be\n\nJob Descript..."


In [331]:
new_df = job_df.merge(job_df2,  left_on=['CompanyName', 'JobTitle', 'Location'], right_on=['Company', 'JobTitle', 'Location'])

In [332]:
engine = create_engine(f"postgresql://postgres:<PASSWORD>@finalproject.cmaigqkcfkxs.us-east-2.rds.amazonaws.com:5432/FinalProject")
new_df.to_sql(name='jobs', con=engine, index=False, if_exists="append")

In [333]:
browser.quit()